In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.utils.data.dataloader import DataLoader
from torch.optim.lr_scheduler import MultiStepLR

import math
import glob
import os
import random
import cv2
import numpy as np
from PIL import Image
from tqdm import tqdm
from torch.utils.data.dataset import Dataset
import xml.etree.ElementTree as ET
import albumentations as albu

torch.backends.cudnn.benchmark = True
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.7' (you have '2.0.6'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


device(type='cuda')

In [2]:
dataset_params = {
  "train_im_sets": ["data/VOC2007", "data/VOC2012"],
  "test_im_sets": ["data/VOC2007-test"],
  "num_classes": 20, # 20 foreground classes
  "im_size": 448
}

model_params = {
  "im_channels": 3,
  "backbone_channels": 512,
  "conv_spatial_size": 7, # Size after all conv layers
  "yolo_conv_channels": 1024,
  "leaky_relu_slope": 0.1,
  "fc_dim": 4096,
  "fc_dropout": 0.5,
  "S": 7, # 49 grid cells
  "B": 2, # 2 boxes per grid cell
  "use_sigmoid": True,
  "use_conv": True
}

train_params = {
  "task_name": "voc",
  "seed": 1111,
  "acc_steps": 1, # Increase if you want to get gradients from >1 steps (kind of mimicking >1 batch size)
  "log_steps": 100,
  "num_epochs": 135,
  "batch_size": 64,
  "lr_steps": [50, 75, 100, 125],
  "lr": 0.001,
  "infer_conf_threshold": 0.2,
  "eval_conf_threshold": 0.001,
  "nms_threshold": 0.5,
  "ckpt_name": "yolo_voc2007.pth"
}

In [3]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [4]:
!unzip -q "/content/drive/MyDrive/Machine Learning/advanced-deep-learning/object-detection/YOLO/data.zip" -d "/content"

In [5]:
def load_images_and_anns(im_sets, label2idx, ann_fname, split):
  """
  Method to get the xml files and for each file
  get all the objects and their ground truth detection
  information for the dataset

  im_sets: Sets of images to consider
  label2idx: Class Name to index mapping for dataset
  ann_fname: txt file containing image names{trainval.txt/test.txt}
  split: train/test
  """
  im_infos = []
  ims = []

  for im_set in im_sets:
    im_names = []

    # Fetch all image names in txt file for this imageset
    for line in open(os.path.join(im_set, "ImageSets", "Main", f"{ann_fname}.txt")):
      im_names.append(line.strip())

    # Set annotation and image path
    ann_dir = os.path.join(im_set, "Annotations")
    im_dir = os.path.join(im_set, "JPEGImages")

    for im_name in im_names:
      ann_file = os.path.join(ann_dir, f"{im_name}.xml")
      im_info = {}
      ann_info = ET.parse(ann_file)
      root = ann_info.getroot()
      size = root.find("size")

      width = int(size.find("width").text)
      height = int(size.find("height").text)

      im_info["img_id"] = os.path.basename(ann_file).split(".xml")[0]
      im_info["filename"] = os.path.join(im_dir, f"{im_info['img_id']}.jpg")

      im_info["width"] = width
      im_info["height"] = height
      detections = []

      # We will keep an image only if there are valid rois in it
      any_valid_object = False
      for obj in ann_info.findall("object"):
        det = {}

        label = label2idx[obj.find("name").text]
        difficult = int(obj.find("difficult").text)
        bbox_info = obj.find("bndbox")

        # Boxes are in x_1, y_1, x_2, y_2 format
        bbox = [
            int(float(bbox_info.find("xmin").text))-1,
            int(float(bbox_info.find("ymin").text))-1,
            int(float(bbox_info.find("xmax").text))-1,
            int(float(bbox_info.find("ymax").text))-1
        ]

        det["label"] = label
        det["bbox"] = bbox
        det["difficult"] = difficult

        # Ignore difficult rois during training
        # At test time eval does the job of ignoring difficult examples.
        if difficult == 0 or split == "test":
          detections.append(det)
          any_valid_object = True

      if any_valid_object:
        im_info["detections"] = detections
        im_infos.append(im_info)

  print(f"Total {len(im_infos)} images found")
  return im_infos

In [12]:
class VOCDataset(Dataset):
  def __init__(self, split, im_sets, im_size=448, S=7, B=2, C=20):
    self.split = split
    # ImageSets for this dataset instance (VOC2007/VOC2007+VOC2012/VOC2007-test)
    self.im_sets = im_sets
    self.fname = "trainval" if self.split == "train" else "test"
    self.im_size = im_size
    # Grid size, B and C parameter for target setting
    self.S = S
    self.B = B
    self.C = C

    # Train and test augmentations
    self.transforms = {
        # Training Augmentations: Horizontal Flip, Affine, ColorJitter
        "train": albu.Compose([
            albu.HorizontalFlip(p=0.5),
            albu.Affine(
                scale=(0.8, 1.2),
                translate_percent=(-0.2, 0.2),
                always_apply=True
            ),
            albu.ColorJitter(
                brightness=(0.8, 1.2),
                contrast=(0.8, 1.2),
                saturation=(0.8, 1.2),
                hue=(-0.2, 0.2),
                always_apply=False,
                p=0.5,
            ),
            albu.Resize(self.im_size, self.im_size)],
            bbox_params=albu.BboxParams(format="pascal_voc",
                                        label_fields=["labels"])),
        "test": albu.Compose([
            albu.Resize(self.im_size, self.im_size),
            ],
            bbox_params=albu.BboxParams(format="pascal_voc",
                                        label_fields=["labels"]))
    }

    classes = [
        "person", "bird", "cat", "cow", "dog", "horse", "sheep",
        "aeroplane", "bicycle", "boat", "bus", "car", "motorbike", "train",
        "bottle", "chair", "diningtable", "pottedplant", "sofa", "tvmonitor"
    ]
    classes = sorted(classes)

    self.label2idx = {classes[idx]: idx for idx in range(len(classes))}
    self.idx2label = {idx: classes[idx] for idx in range(len(classes))}

    print(self.idx2label)

    self.images_info = load_images_and_anns(self.im_sets,
                                            self.label2idx,
                                            self.fname,
                                            self.split)

  def __len__(self):
    return len(self.images_info)

  def __getitem__(self, index):
    im_info = self.images_info[index]
    im = cv2.imread(im_info["filename"])
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)

    # Get annotations for this image
    bboxes = [detection["bbox"] for detection in im_info["detections"]]
    labels = [detection["label"] for detection in im_info["detections"]]
    difficult = [detection["difficult"] for detection in im_info["detections"]]

    # Transform Image and ann according to augmentations list
    transformed_info = self.transforms[self.split](image=im,
                                                    bboxes=bboxes,
                                                    labels=labels)
    im = transformed_info["image"]
    bboxes = torch.as_tensor(transformed_info["bboxes"])
    labels = torch.as_tensor(transformed_info["labels"])
    difficult = torch.as_tensor(difficult)

    # Convert image to tensor and normalize for ResNet pretrained on ImageNet
    im_tensor = torch.from_numpy(im / 255.).permute((2, 0, 1)).float()
    im_tensor_channel_0 = (torch.unsqueeze(im_tensor[0], 0) - 0.485) / 0.229
    im_tensor_channel_1 = (torch.unsqueeze(im_tensor[1], 0) - 0.456) / 0.224
    im_tensor_channel_2 = (torch.unsqueeze(im_tensor[2], 0) - 0.406) / 0.225

    im_tensor = torch.cat((im_tensor_channel_0,
                            im_tensor_channel_1,
                            im_tensor_channel_2), 0)

    bboxes_tensor = torch.as_tensor(bboxes)
    labels_tensor = torch.as_tensor(labels)

    # Build Target for Yolo
    # S x S x (5B +C)
    target_dim = 5 * self.B + self.C
    h, w = im.shape[:2]
    yolo_targets = torch.zeros(self.S, self.S, target_dim)

    # Height and width of grid cells is H // S
    cell_pixels = h // self.S

    if len(bboxes) > 0:
      # Convert x_1, y_1, x_2, y_2 to center_x, center_y, width, height format
      box_widths = bboxes_tensor[:, 2] - bboxes_tensor[:, 0]
      box_heights = bboxes_tensor[:, 3] - bboxes_tensor[:, 1]
      box_center_x = bboxes_tensor[:, 0] + 0.5 * box_widths
      box_center_y = bboxes_tensor[:, 1] + 0.5 * box_heights

      # Get the grid cell i, j from x_center, y_center
      box_i = torch.floor(box_center_x / cell_pixels).long()
      box_j = torch.floor(box_center_y / cell_pixels).long()

      # x_center and y_center offsets from the top left of grid cell and normalized to [0, 1]
      box_xc_cell_offset = (box_center_x - box_i*cell_pixels) / cell_pixels
      box_yc_cell_offset = (box_center_y - box_j*cell_pixels) / cell_pixels

      # width and height targets normalized to [0, 1]
      box_w_label = box_widths / w
      box_h_label = box_heights / h

      # Update the target array for all bboxes
      for idx, b in enumerate(range(bboxes_tensor.size(0))):
        # Make target of the exact same shape as prediction
        for k in range(self.B):
          s = 5 * k

          # target_ij = [xc_offset,yc_offset,sqrt(w),sqrt(h), conf, cls_label]
          yolo_targets[box_j[idx], box_i[idx], s] = box_xc_cell_offset[idx]
          yolo_targets[box_j[idx], box_i[idx], s+1] = box_yc_cell_offset[idx]

          # Use the square root for localization loss function
          yolo_targets[box_j[idx], box_i[idx], s+2] = box_w_label[idx].sqrt()
          yolo_targets[box_j[idx], box_i[idx], s+3] = box_h_label[idx].sqrt()
          yolo_targets[box_j[idx], box_i[idx], s+4] = 1.0

        label = int(labels[b])
        cls_target = torch.zeros((self.C,))
        cls_target[label] = 1.
        yolo_targets[box_j[idx], box_i[idx], 5 * self.B:] = cls_target

    # For training, we use yolo_targets(xoffset, yoffset, sqrt(w), sqrt(h))
    # For evaluation we use bboxes_tensor (x1, y1, x2, y2)
    # Normalize bboxes tensor to be between [0, 1] as thats what evaluation script expects so (x1/w, y1/h, x2/w, y2/h)
    if len(bboxes) > 0:
      bboxes_tensor /= torch.Tensor([[w, h, w, h]]).expand_as(bboxes_tensor)

    targets = {
        "bboxes": bboxes_tensor,
        "labels": labels_tensor,
        "yolo_targets": yolo_targets,
        "difficult": difficult,
    }

    return im_tensor, targets, im_info["filename"]

In [13]:
class YOLOV1(nn.Module):
  """
  Model with three components:
  1. Backbone of ResnNt34 pretrained on 224x224 images from Imagenet
  2. 4 Conv, BatchNorm, and LeakyReLU layers for Yolo Detection Head
  3. Linear layers with final layer having S * S * (5B + C) output dimensions

  The final layer predicts the x_offset_boxB, y_offset_boxB, sqrt_w_boxB, sqrt_h_boxB, conf_boxB,
  and class conditional probabilities for each S * S grid cell
  """
  def __init__(self, im_size, num_classes, model_config):
    super().__init__()

    self.im_size = im_size # 448 x 448
    self.im_channels = model_config["im_channels"] # 3
    self.backbone_channels = model_config["backbone_channels"] # ResNet34 gives feature map with 512 channels
    self.yolo_conv_channels = model_config["yolo_conv_channels"] # 1024 channels
    self.conv_spatial_size = model_config["conv_spatial_size"] # size of final feature map after conv layers
    self.leaky_relu_slope = model_config["leaky_relu_slope"]
    self.yolo_fc_hidden_dim = model_config["fc_dim"]
    self.yolo_fc_dropout_prob = model_config["fc_dropout"]
    self.use_conv = model_config["use_conv"]
    self.S = model_config["S"] # 7
    self.B = model_config["B"] # 2
    self.C = num_classes

    backbone = torchvision.models.resnet34(weights=torchvision.models.ResNet34_Weights.IMAGENET1K_V1)

    # Backbone Layers
    self.features = nn.Sequential(
        backbone.conv1,
        backbone.bn1,
        backbone.relu,
        backbone.maxpool,
        backbone.layer1,
        backbone.layer2,
        backbone.layer3,
        backbone.layer4,
    ) # ResNet34 layers without last linear layers

    # Detection Convolutional Layers
    self.conv_yolo_layers = nn.Sequential(
        nn.Conv2d(in_channels=self.backbone_channels,
                  out_channels=self.yolo_conv_channels,
                  kernel_size=3,
                  stride=1,
                  padding=1,
                  bias=False),
        nn.BatchNorm2d(self.yolo_conv_channels),
        nn.LeakyReLU(self.leaky_relu_slope),

        nn.Conv2d(in_channels=self.yolo_conv_channels,
                  out_channels=self.yolo_conv_channels,
                  kernel_size=3,
                  stride=2,
                  padding=1,
                  bias=False),
        nn.BatchNorm2d(self.yolo_conv_channels),
        nn.LeakyReLU(self.leaky_relu_slope),

        nn.Conv2d(in_channels=self.yolo_conv_channels,
                  out_channels=self.yolo_conv_channels,
                  kernel_size=3,
                  stride=1,
                  padding=1,
                  bias=False),
        nn.BatchNorm2d(self.yolo_conv_channels),
        nn.LeakyReLU(self.leaky_relu_slope),

        nn.Conv2d(in_channels=self.yolo_conv_channels,
                  out_channels=self.yolo_conv_channels,
                  kernel_size=3,
                  stride=1,
                  padding=1,
                  bias=False),
        nn.BatchNorm2d(self.yolo_conv_channels),
        nn.LeakyReLU(self.leaky_relu_slope),
    )

    # Detection Layers
    if self.use_conv:
      self.linear_yolo_layers = nn.Sequential(
          nn.Conv2d(in_channels=self.yolo_conv_channels,
                    out_channels=(5 * self.B + self.C),
                    kernel_size=1,
                    stride=1,
                    padding=0)
      )
    else:
      self.linear_yolo_layers = nn.Sequential(
          nn.Flatten(),
          nn.Linear(in_features=(self.conv_spatial_size * self.conv_spatial_size * self.yolo_conv_channels),
                    out_features=self.yolo_fc_hidden_dim),
          nn.LeakyReLU(self.leaky_relu_slope),
          nn.Dropout(self.yolo_fc_dropout_prob),
          nn.Linear(in_features=self.yolo_fc_hidden_dim,
                    out_features=(self.S * self.S * (5 * self.B + self.C)))
      )

  def forward(self, x):
    # Backbone
    out = self.features(x)

    out = self.conv_yolo_layers(out)
    out = self.linear_yolo_layers(out)

    if self.use_conv:
      # Reshape to (batch_size, S, S, 5 * B + C)
      out = out.permute(0, 2, 3, 1)

    return out

In [14]:
def get_iou(boxes1, boxes2):
  """IOU between two sets of boxes"""

  # Area of boxes (x_2 - x_1) * (y_2 - y_1)
  area1 = (boxes1[..., 2] - boxes1[..., 0]) * (boxes1[..., 3] - boxes1[..., 1])
  area2 = (boxes2[..., 2] - boxes2[..., 0]) * (boxes2[..., 3] - boxes2[..., 1])

  # Get top left coordinates: (x_1, y_1)
  x_left = torch.max(boxes1[..., 0], boxes2[..., 0])
  y_top = torch.max(boxes1[..., 1], boxes2[..., 1])

  # Get bottom right coordinates: (x_2, y_2)
  x_right = torch.min(boxes1[..., 2], boxes2[..., 2])
  y_bottom = torch.min(boxes1[..., 3], boxes2[..., 3])

  intersection_area = (x_right - x_left).clamp(min=0) * (y_bottom - y_top).clamp(min=0)
  union = area1.clamp(min=0) + area2.clamp(min=0) - intersection_area

  iou = intersection_area / (union + 1E-6)
  return iou

In [15]:
class YOLOV1Loss(nn.Module):
  """
  Loss module for YoloV1 which caters to the following components:
  1. Localization Loss for responsible predictor boxes
  2. Confidence Loss for responsible predictor boxes
  2. Confidence Loss for non-responsible predictor boxes of cells assigned with objects
  2. Confidence Loss for ALL predictor boxes of cells not assigned with objects
  3. Classification Loss
  """
  def __init__(self, S=7, B=2, C=20):
    super().__init__()

    self.S = S
    self.B = B
    self.C = C
    self.lambda_coord = 5
    self.lambda_noobj = 0.5

  def forward(self, preds, targets, use_sigmoid=False):
    """
    Compute the YOLO Loss
    The target element for each cell has been duplicated 5B times(done in VOCDataset)

    preds: (batch_size, S * S * (5B+C)) tensor
    targets: (batch_size, S, S, (5B + C)) tensor
    use_sigmoid: Whether to use sigmoid activation for box predicitons or not
    """
    batch_size = preds.size(0)

    # Reshape preds to same shape as targets
    preds = preds.reshape(batch_size, self.S, self.S, 5 * self.B + self.C)
    # preds shape: (batch_size, S, S, 5B + C)

    # Sigmoid leads to quicker convergence
    if use_sigmoid:
      preds[..., :5 * self.B] = torch.sigmoid(preds[..., :5 * self.B])

    # For localization and confidence loss, we need to get the responsible predictor box for each grid cell

    # Define shifts for all grid cell locations and normalize between 0 and 1
    # Will use these for converting x_center_offset and y_center_offset
    # Values for x_1, y_2, x_2, and y_2 are normalized between [0, 1]
    # S cells = 1 => each cell adds 1/S pixels of shift
    shifts_x = torch.arange(0,
                            self.S,
                            dtype=torch.int32,
                            device=preds.device) * 1 / float(self.S)
    shifts_y = torch.arange(0,
                            self.S,
                            dtype=torch.int32,
                            device=preds.device) * 1 / float(self.S)

    # Create a grid using these shifts
    shifts_y, shifts_x = torch.meshgrid(shifts_y, shifts_x, indexing="ij")

    # shifts shape: (1, S, S, B)
    shifts_x = shifts_x.reshape((1, self.S, self.S, 1)).repeat(1, 1, 1, self.B)
    shifts_y = shifts_y.reshape((1, self.S, self.S, 1)).repeat(1, 1, 1, self.B)

    # pred_boxes shape: (batch_size, S, S, B, 5)
    pred_boxes = preds[..., :5 * self.B].reshape(batch_size, self.S, self.S, self.B, -1)

    # xc_offset, yc_offset, width, height -> x_1, y_1, x_2, y_2 normalized betwen [0, 1]
    # x_center = (xc_offset / S + shift_x)
    # x_1 = x_center - 0.5 * w
    # x_2 = x_center + 0.5 * w

    # Use squared value because the model predicts the square root of width and height
    pred_boxes_x1 = ((pred_boxes[..., 0] / self.S + shifts_x) - 0.5 * torch.square(pred_boxes[..., 2]))
    pred_boxes_x1 = pred_boxes_x1.unsqueeze(dim=-1)

    pred_boxes_y1 = ((pred_boxes[..., 1] / self.S + shifts_y) - 0.5 * torch.square(pred_boxes[..., 3]))
    pred_boxes_y1 = pred_boxes_y1.unsqueeze(dim=-1)

    pred_boxes_x2 = ((pred_boxes[..., 0] / self.S + shifts_x) + 0.5 * torch.square(pred_boxes[..., 2]))
    pred_boxes_x2 = pred_boxes_x2.unsqueeze(dim=-1)

    pred_boxes_y2 = ((pred_boxes[..., 1] / self.S + shifts_y) + 0.5 * torch.square(pred_boxes[..., 3]))
    pred_boxes_y2 = pred_boxes_y2.unsqueeze(dim=-1)

    pred_boxes_x1y1x2y2 = torch.cat([
        pred_boxes_x1,
        pred_boxes_y1,
        pred_boxes_x2,
        pred_boxes_y2], dim=-1)

    # Do the same for the target boxes
    # target_boxes shape: (batch_size, S, S, B, 5)
    target_boxes = targets[..., :5*self.B].reshape(batch_size, self.S, self.S, self.B, -1)

    target_boxes_x1 = ((target_boxes[..., 0] / self.S + shifts_x) - 0.5 * torch.square(target_boxes[..., 2]))
    target_boxes_x1 = target_boxes_x1.unsqueeze(dim=-1)

    target_boxes_y1 = ((target_boxes[..., 1] / self.S + shifts_y) - 0.5 * torch.square(target_boxes[..., 3]))
    target_boxes_y1 = target_boxes_y1.unsqueeze(dim=-1)

    target_boxes_x2 = ((target_boxes[..., 0] / self.S + shifts_x) + 0.5 * torch.square(target_boxes[..., 2]))
    target_boxes_x2 = target_boxes_x2.unsqueeze(dim=-1)

    target_boxes_y2 = ((target_boxes[..., 1] / self.S + shifts_y) + 0.5 * torch.square(target_boxes[..., 3]))
    target_boxes_y2 = target_boxes_y2.unsqueeze(dim=-1)

    target_boxes_x1y1x2y2 = torch.cat([
        target_boxes_x1,
        target_boxes_y1,
        target_boxes_x2,
        target_boxes_y2], dim=-1)

    # pred_boxes_x1y1x2y2 shape: (batch_size, S, S, B, 4)
    # target_boxes_x1y1x2y2 shape: (batch_size, S, S, B, 4)

    # iou shape: (batch_size, S, S, B)
    iou = get_iou(pred_boxes_x1y1x2y2, target_boxes_x1y1x2y2)

    # Get the max along the last dimension which is the box index that has the maximum IOU at every grid cell
    # max_iou_val and max_iou_idx shape: (batch_size, S, S, 1)
    max_iou_val, max_iou_idx = iou.max(dim=-1, keepdim=True)

    # For localization and confidence loss, we need to find the cells that have objects assigned and which are the responsible predictor boxes

    # Indicator Definitions
    max_iou_idx = max_iou_idx.repeat(1, 1, 1, self.B) # (batch_size, S, S, 1) -> (batch_size, S, S, B)

    bb_idxs = (torch.arange(self.B).reshape(1, 1, 1, self.B).expand_as(max_iou_idx).to(preds.device)) # (batch_size, S, S, B)

    # Only the index which of the box that has the highest IOU will be 1, the others will be 0
    is_max_iou_box = (max_iou_idx == bb_idxs).long() # (batch_size, S, S, B)

    obj_indicator = targets[..., 4:5] # (batch_size, S, S, 1)

    # Classification Loss
    cls_target = targets[..., 5 * self.B:]
    cls_preds = preds[..., 5 * self.B:]
    cls_mse = (cls_preds - cls_target)**2

    # Only keep losses from cells with objects assigned
    cls_mse = (obj_indicator * cls_mse).sum()

    # Confidence Loss for the responsible predictor boxes

    # Filters out the cells which were not assigned an object and boxes that are not the responsible predictor
    is_max_box_obj_indicator = is_max_iou_box * obj_indicator
    obj_mse = (pred_boxes[..., 4] - max_iou_val)**2

    # Only keep losses from boxes of cells with object assigned and that box which is the responsible predictor
    obj_mse = (is_max_box_obj_indicator * obj_mse).sum()

    # Localization Loss for x, y, w, h
    x_mse = (pred_boxes[..., 0] - target_boxes[..., 0])**2
    y_mse = (pred_boxes[..., 1] - target_boxes[..., 1])**2
    w_sqrt_mse = (pred_boxes[..., 2] - target_boxes[..., 2])**2
    h_sqrt_mse = (pred_boxes[..., 3] - target_boxes[..., 3])**2

    # Only keep losses from boxes of cells with object assigned and that box which is the responsible predictor
    x_mse = (is_max_box_obj_indicator * x_mse).sum()
    y_mse = (is_max_box_obj_indicator * y_mse).sum()
    w_sqrt_mse = (is_max_box_obj_indicator * w_sqrt_mse).sum()
    h_sqrt_mse = (is_max_box_obj_indicator * h_sqrt_mse).sum()

    # Confidence Loss for background boxes and cells
    # Boxes of cells assigned with object that aren't responsible predictor boxes and for boxes of cell not assigned with object
    no_object_indicator = 1 - is_max_box_obj_indicator
    no_obj_mse = (pred_boxes[..., 4] - torch.zeros_like(pred_boxes[..., 4])) ** 2
    no_obj_mse = (no_object_indicator * no_obj_mse).sum()

    # Total Loss
    loss = self.lambda_coord * (x_mse + y_mse + w_sqrt_mse + h_sqrt_mse)
    loss += cls_mse + obj_mse
    loss += self.lambda_noobj * no_obj_mse
    loss = loss / batch_size
    return loss

In [16]:
def collate_function(data):
  return list(zip(*data))

In [18]:
def train():
  dataset_config = dataset_params
  model_config = model_params
  train_config = train_params

  # Set the random seed
  seed = train_config["seed"]
  torch.manual_seed(seed)
  np.random.seed(seed)
  random.seed(seed)
  if device.type == "cuda":
    torch.cuda.manual_seed_all(seed)

  voc = VOCDataset("train", im_sets=dataset_config["train_im_sets"])
  train_dataset = DataLoader(voc,
                             batch_size=train_config["batch_size"],
                             shuffle=True,
                             collate_fn=collate_function,
                             num_workers=os.cpu_count(),
                             pin_memory=True)

  yolo_model = YOLOV1(im_size=dataset_config["im_size"],
                      num_classes=dataset_config["num_classes"],
                      model_config=model_config)
  yolo_model.train()
  yolo_model.to(device)

  if not os.path.exists(train_config["task_name"]):
    os.mkdir(train_config["task_name"])

  optimizer = torch.optim.SGD(lr=train_config["lr"],
                              params=filter(lambda p: p.requires_grad,
                                            yolo_model.parameters()),
                              weight_decay=5E-4,
                              momentum=0.9)

  scheduler = MultiStepLR(optimizer, milestones=train_config["lr_steps"], gamma=0.5)
  criterion = YOLOV1Loss()
  acc_steps = train_config["acc_steps"]
  num_epochs = train_config["num_epochs"]
  steps = 0

  # Resume training from a saved checkpoint if it exists
  checkpoint_path = os.path.join(train_config["task_name"], train_config["ckpt_name"])
  start_epoch = 0

  if os.path.exists(checkpoint_path):
    print("Resuming training from checkpoint...")
    checkpoint = torch.load(checkpoint_path, map_location=device)

    yolo_model.load_state_dict(checkpoint["model_state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
    scheduler.load_state_dict(checkpoint["scheduler_state_dict"])
    start_epoch = checkpoint["epoch"]
    steps = checkpoint["steps"]

  optimizer.zero_grad()

  for epoch_idx in range(start_epoch, num_epochs):
    losses = []
    optimizer.zero_grad()
    for idx, (ims, targets, _) in enumerate(tqdm(train_dataset)):
      yolo_targets = torch.cat([
          target["yolo_targets"].unsqueeze(0).float().to(device)
          for target in targets
      ], dim=0)

      im = torch.cat([im.unsqueeze(0).float().to(device) for im in ims], dim=0)

      yolo_preds = yolo_model(im)

      loss = criterion(yolo_preds, yolo_targets, use_sigmoid=model_config["use_sigmoid"])
      loss = loss / acc_steps
      loss.backward()
      losses.append(loss.item())

      if (idx + 1) % acc_steps == 0:
        optimizer.step()
        optimizer.zero_grad()
      if steps % train_config["log_steps"] == 0:
        print(f"Loss: {np.mean(losses):.4f}")
      if torch.isnan(loss):
        print("Loss is becoming nan. Exiting")
        exit(0)

      steps += 1
    print(f"Finished epoch {epoch_idx + 1} | Loss: {np.mean(losses):.4f}")
    optimizer.step()
    optimizer.zero_grad()
    scheduler.step()

    checkpoint = {
        "epoch": epoch_idx,
        "steps": steps,
        "model_state_dict": yolo_model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
        "scheduler_state_dict": scheduler.state_dict(),
    }
    torch.save(checkpoint, os.path.join(train_config["task_name"], train_config["ckpt_name"]))

  print("Done Training...")

train()

<ipython-input-12-16ecad10404b>:18: UserWarning: Argument(s) 'always_apply' are not valid for transform Affine
  albu.Affine(
<ipython-input-12-16ecad10404b>:23: UserWarning: Argument(s) 'always_apply' are not valid for transform ColorJitter
  albu.ColorJitter(


{0: 'aeroplane', 1: 'bicycle', 2: 'bird', 3: 'boat', 4: 'bottle', 5: 'bus', 6: 'car', 7: 'cat', 8: 'chair', 9: 'cow', 10: 'diningtable', 11: 'dog', 12: 'horse', 13: 'motorbike', 14: 'person', 15: 'pottedplant', 16: 'sheep', 17: 'sofa', 18: 'train', 19: 'tvmonitor'}
Total 16551 images found


  0%|          | 1/259 [00:04<19:30,  4.54s/it]

Loss: 30.8514


 39%|███▉      | 101/259 [00:27<00:45,  3.48it/s]

...

Loss: 0.4057


 75%|███████▌  | 195/259 [00:48<00:14,  4.33it/s]

Loss: 0.4176


100%|██████████| 259/259 [01:02<00:00,  4.16it/s]


Finished epoch 135 | Loss: 0.4206
Done Training...


In [19]:
COLORS = [
    [0, 0, 0],
    [200, 0, 0],
    [0, 200, 0],
    [50, 128, 0],
    [0, 0, 128],
    [128, 0, 128],
    [0, 128, 128],
    [128, 128, 128],
    [50, 0, 0],
    [192, 0, 0],
    [64, 128, 0],
    [192, 128, 0],
    [64, 0, 128],
    [192, 0, 128],
    [50, 128, 200],
    [192, 128, 200],
    [0, 64, 0],
    [128, 64, 0],
    [0, 192, 0],
    [128, 192, 0],
    [0, 64, 128]
]


def visualize_bbox(img, bbox, class_name, score=None, color=(255, 0, 0), thickness=2):
    """
    Draws a single bounding box on image
    """
    x_min, y_min, x_max, y_max = bbox
    x_min, y_min, x_max, y_max = int(x_min), int(y_min), int(x_max), int(y_max )

    cv2.rectangle(img, (x_min, y_min), (x_max, y_max), color=color, thickness=thickness)
    box_text = class_name + ' {:.2f}'.format(score) if score is not None else class_name
    ((text_width, text_height), _) = cv2.getTextSize(box_text, cv2.FONT_HERSHEY_SIMPLEX, 0.45, 1)
    cv2.rectangle(img, (x_min, y_min - int(1.3 * text_height)), (x_min + text_width, y_min), color, -1)

    cv2.putText(
        img,
        text=box_text,
        org=(x_min, y_min - int(0.3 * text_height)),
        fontFace=cv2.FONT_HERSHEY_SIMPLEX,
        fontScale=0.45,
        color=(255, 255, 255),
        lineType=cv2.LINE_AA,
    )

    return img


def visualize(image, bboxes, category_ids, category_id_to_name, scores=None):
    img = image.copy()
    for idx, (bbox, category_id) in enumerate(zip(bboxes, category_ids)):
      class_name = category_id_to_name[category_id]
      img = visualize_bbox(img, bbox, class_name, scores[idx] if scores is not None else None)

    return img


def draw_grid(img, grid_shape, color=(0, 0, 0), thickness=2):
    """
    Draws a grid on image
    """
    grid_im = np.copy(img)
    h, w, _ = grid_im.shape
    rows, cols = grid_shape
    dy, dx = h / rows, w / cols

    # draw vertical lines
    for x in np.linspace(start=dx, stop=w-dx, num=cols-1):
      x = int(round(x))
      cv2.line(grid_im, (x, 0), (x, h), color=color, thickness=thickness)

    # draw horizontal lines
    for y in np.linspace(start=dy, stop=h-dy, num=rows-1):
      y = int(round(y))
      cv2.line(grid_im, (0, y), (w, y), color=color, thickness=thickness)

    return grid_im


def draw_cls_grid(img, cls_idx, grid_shape):
  """
  Draws color coded grid for the entire image
  coded based on the class label
  """
  rect_im = np.copy(img)
  h, w, _ = rect_im.shape
  rows, cols = grid_shape
  dy, dx = h / rows, w / cols
  for i in range(rows):
    for j in range(cols):
          cv2.rectangle(rect_im, (int(i*dx), int(j*dy)), (int((i+1)*dx), int((j+1)*dy)),
                        thickness=-1,
                        color=COLORS[cls_idx[j, i].item()])

  return rect_im


def draw_cls_text(img, cls_idx, cls_idx_label, grid_shape):
  """
  Writes class text name in grid center locations
  """
  rect_im = np.copy(img)
  h, w, _ = rect_im.shape
  rows, cols = grid_shape
  dy, dx = h / rows, w / cols
  for i in range(rows):
    for j in range(cols):
      cls_label = cls_idx_label[cls_idx[j, i].item()]
      cv2.putText(rect_im,
                  cls_label[:6],
                  (int((i+0.1)*dx), int((j+0.5)*dy)),
                  fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                  fontScale=0.45,
                  color=(255, 255, 255),
                  lineType=cv2.LINE_AA)
  return rect_im

In [20]:
def get_iou_map(det, gt):
  det_x1, det_y1, det_x2, det_y2 = det
  gt_x1, gt_y1, gt_x2, gt_y2 = gt

  x_left = max(det_x1, gt_x1)
  y_top = max(det_y1, gt_y1)
  x_right = min(det_x2, gt_x2)
  y_bottom = min(det_y2, gt_y2)

  if x_right < x_left or y_bottom < y_top:
    return 0.0

  area_intersection = (x_right - x_left) * (y_bottom - y_top)
  det_area = (det_x2 - det_x1) * (det_y2 - det_y1)
  gt_area = (gt_x2 - gt_x1) * (gt_y2 - gt_y1)

  area_union = float(det_area + gt_area - area_intersection + 1E-6)
  iou = area_intersection / area_union

  return iou

In [33]:
def compute_map(det_boxes, gt_boxes, iou_threshold=0.5, method="area", difficult=None):
  gt_labels = {cls_key for im_gt in gt_boxes for cls_key in im_gt.keys()}
  gt_labels = sorted(gt_labels)

  all_aps = {}

  # Average precisions for ALL classes
  aps = []
  for idx, label in enumerate(gt_labels):
    # Get detection predictions of this class
    cls_dets = [
        [im_idx, im_dets_label] for im_idx, im_dets in enumerate(det_boxes)
        if label in im_dets for im_dets_label in im_dets[label]
    ]

    # Sort them by confidence score
    cls_dets = sorted(cls_dets, key=lambda k: -k[1][-1])

    # For tracking which gt boxes of this class have already been matched
    gt_matched = [[False for _ in im_gts[label]] for im_gts in gt_boxes]
    # Number of gt boxes for this class for recall calculation
    num_gts = sum([len(im_gts[label]) for im_gts in gt_boxes])
    num_difficults = sum([sum(difficults_label[label]) for difficults_label in difficult])

    tp = [0] * len(cls_dets)
    fp = [0] * len(cls_dets)

    # For each prediction
    for det_idx, (im_idx, det_pred) in enumerate(cls_dets):
      # Get gt boxes for this image and this label
      im_gts = gt_boxes[im_idx][label]
      im_gt_difficults = difficult[im_idx][label]

      max_iou_found = -1
      max_iou_gt_idx = -1

      # Get best matching gt box
      for gt_box_idx, gt_box in enumerate(im_gts):
        gt_box_iou = get_iou_map(det_pred[:-1], gt_box)
        if gt_box_iou > max_iou_found:
          max_iou_found = gt_box_iou
          max_iou_gt_idx = gt_box_idx
      # TP only if iou >= threshold and this gt has not yet been matched
      if max_iou_found >= iou_threshold:
        if not im_gt_difficults[max_iou_gt_idx]:
          if not gt_matched[im_idx][max_iou_gt_idx]:
            # If tp then we set this gt box as matched
            gt_matched[im_idx][max_iou_gt_idx] = True
            tp[det_idx] = 1
          else:
            fp[det_idx] = 1
      else:
        fp[det_idx] = 1

    # Cumulative tp and fp
    tp = np.cumsum(tp)
    fp = np.cumsum(fp)

    eps = np.finfo(np.float32).eps
    # recalls = tp / np.maximum(num_gts, eps)
    recalls = tp / np.maximum(num_gts - num_difficults, eps)
    precisions = tp / np.maximum((tp + fp), eps)

    if method == "area":
      recalls = np.concatenate(([0.0], recalls, [1.0]))
      precisions = np.concatenate(([0.0], precisions, [0.0]))

      # Replace precision values with recall r with maximum precision value
      # of any recall value >= r
      # This computes the precision envelope
      for i in range(precisions.size - 1, 0, -1):
          precisions[i - 1] = np.maximum(precisions[i - 1], precisions[i])
      # For computing area, get points where recall changes value
      i = np.where(recalls[1:] != recalls[:-1])[0]
      # Add the rectangular areas to get ap
      ap = np.sum((recalls[i + 1] - recalls[i]) * precisions[i + 1])
    elif method == "interp":
      ap = 0.0
      for interp_pt in np.arange(0, 1 + 1E-3, 0.1):
        # Get precision values for recall values >= interp_pt
        prec_interp_pt = precisions[recalls >= interp_pt]

        # Get max of those precision values
        prec_interp_pt= prec_interp_pt.max() if prec_interp_pt.size>0.0 else 0.0
        ap += prec_interp_pt
      ap = ap / 11.0
    else:
      raise ValueError("Method can only be area or interp")
    if num_gts > 0:
      aps.append(ap)
      all_aps[label] = ap
    else:
      all_aps[label] = np.nan

  # compute mAP at provided iou threshold
  mean_ap = sum(aps) / len(aps)
  return mean_ap, all_aps

In [30]:
def load_model_and_dataset():
  dataset_config = dataset_params
  model_config = model_params
  train_config = train_params

  voc = VOCDataset("test",
                   im_sets=dataset_config["test_im_sets"],
                   im_size=dataset_config["im_size"],
                   S=model_config["S"],
                   B=model_config["B"],
                   C=dataset_config["num_classes"])

  test_dataset = DataLoader(voc,
                            batch_size=1,
                            shuffle=False,
                            num_workers=os.cpu_count(),
                            pin_memory=True)

  yolo_model = YOLOV1(im_size=dataset_config["im_size"],
                      num_classes=dataset_config["num_classes"],
                      model_config=model_config)
  yolo_model.eval()
  yolo_model.to(device)

  checkpoint = torch.load(os.path.join(train_config["task_name"], train_config["ckpt_name"]),
                          map_location=device)

  yolo_model.load_state_dict(checkpoint["model_state_dict"])

  return yolo_model, voc, test_dataset

In [31]:
def convert_yolo_pred_x1y1x2y2(yolo_pred, S, B, C, use_sigmoid=False):
  """
  Method converts yolo predictions to x_1, y_1, x_2, y_2 format
  """
  out = yolo_pred.reshape((S, S, 5 * B + C))
  if use_sigmoid:
    out[..., :5 * B] = torch.nn.functional.sigmoid(out[..., :5 * B])

  out = torch.clamp(out, min=0., max=1.)
  class_score, class_idx = torch.max(out[..., 5 * B:], dim=-1)

  # Create a grid using these shifts
  # Will use these for converting x_center_offset and y_center_offset
  # values to x_1, y_1, x_2, y_2 that are normalized between [0,1]
  # S cells = 1 => each cell adds 1/S pixels of shift
  shifts_x = torch.arange(0, S, dtype=torch.int32, device=out.device) * 1 / float(S)
  shifts_y = torch.arange(0, S, dtype=torch.int32, device=out.device) * 1 / float(S)
  shifts_y, shifts_x = torch.meshgrid(shifts_y, shifts_x, indexing="ij")

  boxes = []
  confidences = []
  labels = []
  for box_idx in range(B):
    # xc_offset, yc_offset, width, height -> x_1, y_1, x_2, y_2
    boxes_x1 = ((out[..., box_idx * 5] * 1 / float(S) + shifts_x) - 0.5 * torch.square(out[..., 2 + box_idx * 5])).reshape(-1, 1)
    boxes_y1 = ((out[..., 1 + box_idx * 5] * 1 / float(S) + shifts_y) - 0.5 * torch.square(out[..., 3 + box_idx * 5])).reshape(-1, 1)
    boxes_x2 = ((out[..., box_idx * 5] * 1 / float(S) + shifts_x) + 0.5 * torch.square(out[..., 2 + box_idx * 5])).reshape(-1, 1)
    boxes_y2 = ((out[..., box_idx * 5] * 1 / float(S) + shifts_y) + 0.5 * torch.square(out[..., 3 + box_idx * 5])).reshape(-1, 1)

    boxes.append(torch.cat([boxes_x1, boxes_y1, boxes_x2, boxes_y2], dim=-1))
    confidences.append((out[..., 4 + box_idx * 5] * class_score).reshape(-1))
    labels.append(class_idx.reshape(-1))

  boxes = torch.cat(boxes, dim=0)
  scores = torch.cat(confidences, dim=0)
  labels = torch.cat(labels, dim=0)

  return boxes, scores, labels

In [24]:
def infer():
  if not os.path.exists("samples"):
      os.mkdir("samples")

  yolo_model, voc, test_dataset = load_model_and_dataset()
  conf_threshold = train_params["infer_conf_threshold"]
  nms_threshold = train_params["nms_threshold"]

  num_samples = 5

  for i in tqdm(range(num_samples)):
    dataset_idx = random.randint(0, len(voc) - 1)
    im_tensor, targets, fname = voc[dataset_idx]

    out = yolo_model(im_tensor.unsqueeze(0).to(device))

    # Have to convert the output for YOLO format to x_1, y_1, x_2, y_2
    boxes, scores, labels = convert_yolo_pred_x1y1x2y2(out,
                                                       S=yolo_model.S,
                                                       B=yolo_model.B,
                                                       C=yolo_model.C,
                                                       use_sigmoid=model_params["use_sigmoid"])

    # Confidence Score Thresholding
    keep = torch.where(scores > conf_threshold)[0]
    boxes = boxes[keep]
    scores = scores[keep]
    labels = labels[keep]

    # NMS
    keep_mask = torch.zeros_like(scores, dtype=torch.bool)
    for class_id in torch.unique(labels):
      curr_indices = torch.where(labels == class_id)[0]
      curr_keep_indices = torch.ops.torchvision.nms(boxes[curr_indices],
                                                    scores[curr_indices],
                                                    nms_threshold)
      keep_mask[curr_indices[curr_keep_indices]] = True

    keep = torch.where(keep_mask)[0]
    boxes = boxes[keep]
    scores = scores[keep]
    labels = labels[keep]

    # Visualization

    if not os.path.exists("samples/preds"):
      os.mkdir("samples/preds")
    if not os.path.exists("samples/grid_cls"):
      os.mkdir("samples/grid_cls")

    im = cv2.imread(fname)
    h, w = im.shape[:2]
    # Scale prediction boxes x_1, y_1, x_2, y_2 from [0, 1] to 0-w and 0-h
    boxes[..., 0::2] = (w * boxes[..., 0::2])
    boxes[..., 1::2] = (h * boxes[..., 1::2])

    out_img = visualize(image=im,
                        bboxes=boxes.detach().cpu().numpy(),
                        category_ids=labels.detach().cpu().numpy(),
                        category_id_to_name=voc.idx2label,
                        scores=scores.detach().cpu().numpy())

    cv2.imwrite(f"samples/preds/{i}_pred.jpeg", out_img)

    # Below lines of code are only for drawing class prob map
    im = cv2.resize(im, (yolo_model.im_size, yolo_model.im_size))

    # Draw a SxS grid on image
    grid_im = draw_grid(im, (yolo_model.S, yolo_model.S))

    out = out.reshape((yolo_model.S, yolo_model.S, 5 * yolo_model.B + yolo_model.C))
    cls_val, cls_idx = torch.max(out[..., 5 * yolo_model.B:], dim=-1)

    # Draw colored squares for probability mappings on image
    rect_im = draw_cls_grid(im, cls_idx, (yolo_model.S, yolo_model.S))
    # Draw grid again on top of this image
    rect_im = draw_grid(rect_im, (yolo_model.S, yolo_model.S))

    # Overlay image with grid and cls mappings with grid on top of each other
    res = cv2.addWeighted(rect_im, 0.5, grid_im, 0.5, 1.0)

    # Write class labels on grid on this image
    res = draw_cls_text(res, cls_idx, voc.idx2label, (yolo_model.S, yolo_model.S))
    cv2.imwrite(f"samples/grid_cls/{i}_grid_map.jpeg", res)

  print("Done Detecting..")

infer()

<ipython-input-12-16ecad10404b>:18: UserWarning: Argument(s) 'always_apply' are not valid for transform Affine
  albu.Affine(
<ipython-input-12-16ecad10404b>:23: UserWarning: Argument(s) 'always_apply' are not valid for transform ColorJitter
  albu.ColorJitter(


{0: 'aeroplane', 1: 'bicycle', 2: 'bird', 3: 'boat', 4: 'bottle', 5: 'bus', 6: 'car', 7: 'cat', 8: 'chair', 9: 'cow', 10: 'diningtable', 11: 'dog', 12: 'horse', 13: 'motorbike', 14: 'person', 15: 'pottedplant', 16: 'sheep', 17: 'sofa', 18: 'train', 19: 'tvmonitor'}
Total 4952 images found


100%|██████████| 5/5 [00:00<00:00, 10.38it/s]

Done Detecting..


In [34]:
def evaluate_map():
  yolo_model, voc, test_dataset = load_model_and_dataset()

  conf_threshold = train_params["eval_conf_threshold"]
  nms_threshold = train_params["nms_threshold"]

  gts = []
  preds = []
  difficults = []
  for im_tensor, target, fname in tqdm(test_dataset):
    im_tensor = im_tensor.float().to(device)
    target_bboxes = target["bboxes"].float().to(device)[0]
    target_labels = target["labels"].long().to(device)[0]
    difficult = target["difficult"].long().to(device)[0]

    out = yolo_model(im_tensor)

    boxes, scores, labels = convert_yolo_pred_x1y1x2y2(out,
                                                        S=yolo_model.S,
                                                        B=yolo_model.B,
                                                        C=yolo_model.C,
                                                        use_sigmoid=model_params["use_sigmoid"])

    # Confidence Score Thresholding
    keep = torch.where(scores > conf_threshold)[0]
    boxes = boxes[keep]
    scores = scores[keep]
    labels = labels[keep]

    # NMS
    keep_mask = torch.zeros_like(scores, dtype=torch.bool)
    for class_id in torch.unique(labels):
      curr_indices = torch.where(labels == class_id)[0]
      curr_keep_indices = torch.ops.torchvision.nms(boxes[curr_indices],
                                                    scores[curr_indices],
                                                    nms_threshold)

      keep_mask[curr_indices[curr_keep_indices]] = True

    keep = torch.where(keep_mask)[0]
    boxes = boxes[keep]
    scores = scores[keep]
    labels = labels[keep]

    pred_boxes = {}
    gt_boxes = {}
    difficult_boxes = {}

    for label_name in voc.label2idx:
      pred_boxes[label_name] = []
      gt_boxes[label_name] = []
      difficult_boxes[label_name] = []

    for idx, box in enumerate(boxes):
      x1, y1, x2, y2 = box.detach().cpu().numpy()
      label = labels[idx].detach().cpu().item()
      score = scores[idx].detach().cpu().item()
      label_name = voc.idx2label[label]
      pred_boxes[label_name].append([x1, y1, x2, y2, score])

    for idx, box in enumerate(target_bboxes):
      x1, y1, x2, y2 = box.detach().cpu().numpy()
      label = target_labels[idx].detach().cpu().item()
      label_name = voc.idx2label[label]
      gt_boxes[label_name].append([x1, y1, x2, y2])
      difficult_boxes[label_name].append(difficult[idx].detach().cpu().item())

      gts.append(gt_boxes)
      preds.append(pred_boxes)
      difficults.append(difficult_boxes)

  mean_ap, all_aps = compute_map(preds, gts, method="area", difficult=difficults)

  print("Class Wise Average Precisions")
  for idx in range(len(voc.idx2label)):
    print(f"AP for class {voc.idx2label[idx]} = {all_aps[voc.idx2label[idx]]:.4f}")

  print(f"Mean Average Precision: {mean_ap:.4f}")

evaluate_map()

<ipython-input-12-16ecad10404b>:18: UserWarning: Argument(s) 'always_apply' are not valid for transform Affine
  albu.Affine(
<ipython-input-12-16ecad10404b>:23: UserWarning: Argument(s) 'always_apply' are not valid for transform ColorJitter
  albu.ColorJitter(


{0: 'aeroplane', 1: 'bicycle', 2: 'bird', 3: 'boat', 4: 'bottle', 5: 'bus', 6: 'car', 7: 'cat', 8: 'chair', 9: 'cow', 10: 'diningtable', 11: 'dog', 12: 'horse', 13: 'motorbike', 14: 'person', 15: 'pottedplant', 16: 'sheep', 17: 'sofa', 18: 'train', 19: 'tvmonitor'}
Total 4952 images found


100%|██████████| 4952/4952 [01:14<00:00, 66.66it/s]


Class Wise Average Precisions
AP for class aeroplane = 0.3184
AP for class bicycle = 0.4868
AP for class bird = 0.2774
AP for class boat = 0.2727
AP for class bottle = 0.1549
AP for class bus = 0.5753
AP for class car = 0.4371
AP for class cat = 0.7185
AP for class chair = 0.2702
AP for class cow = 0.4358
AP for class diningtable = 0.4733
AP for class dog = 0.6572
AP for class horse = 0.6561
AP for class motorbike = 0.4671
AP for class person = 0.4352
AP for class pottedplant = 0.1607
AP for class sheep = 0.4112
AP for class sofa = 0.4401
AP for class train = 0.7036
AP for class tvmonitor = 0.3888
Mean Average Precision: 0.4370


In [27]:
!zip -r samples.zip samples

  adding: samples/ (stored 0%)
  adding: samples/grid_cls/ (stored 0%)
  adding: samples/grid_cls/2_grid_map.jpeg (deflated 5%)
  adding: samples/grid_cls/3_grid_map.jpeg (deflated 1%)
  adding: samples/grid_cls/1_grid_map.jpeg (deflated 1%)
  adding: samples/grid_cls/4_grid_map.jpeg (deflated 3%)
  adding: samples/grid_cls/0_grid_map.jpeg (deflated 1%)
  adding: samples/preds/ (stored 0%)
  adding: samples/preds/0_pred.jpeg (deflated 0%)
  adding: samples/preds/2_pred.jpeg (deflated 0%)
  adding: samples/preds/1_pred.jpeg (deflated 0%)
  adding: samples/preds/3_pred.jpeg (deflated 0%)
  adding: samples/preds/4_pred.jpeg (deflated 1%)


In [28]:
from google.colab import files

files.download("samples.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>